In [3]:
from sqlalchemy import create_engine
import pandas as pd

#### Crear el "engine" pasando la dirección de la db

In [5]:
engine = create_engine('postgresql://postgres@localhost:5432/Vinos')
engine

Engine(postgresql://postgres@localhost:5432/Vinos)

#### Lleno la tabla de Clientes

In [6]:
data = [[1, 'Customer 1', 'Address 1', '11111111', '2000-10-10', 'F'],
        [2, 'Customer 2', 'Address 2', '22222222', '2015-01-12', 'M'],
        [3, 'Customer 3', 'Address 3', '33333333', '2014-01-10', 'F'],
        [4, 'Customer 3', 'Address 4', '44444444', '2011-01-10', 'F']
       ]

In [11]:
df_customer = pd.read_sql_query('select * from "Customer"',con=engine)
df_customer

,idCustomer,name,addess,phone,birthday,gender
0,1,Customer 1,Address 1,11111111,2000-10-10,F
1,2,Customer 2,Address 2,22222222,2015-01-12,M
2,3,Customer 3,Address 3,33333333,2014-01-10,F
3,4,Customer 3,Address 4,44444444,2011-01-10,F


In [2]:
new_df = pd.DataFrame(data, columns=df_customer.columns)

NameError: name 'pd' is not defined

In [1]:
new_df.to_sql('Customer', engine, if_exists='append', index=None, chunksize=None)

NameError: name 'new_df' is not defined

In [8]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 6 columns):
idCustomer    4 non-null int64
name          4 non-null object
addess        4 non-null object
phone         4 non-null object
birthday      4 non-null datetime64[ns]
gender        4 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 224.0+ bytes


#### Creo un JSON con los datos

In [27]:
json_customer = df_customer.to_json('customer.json',date_unit='ns')
csv_customer = df_customer.to_csv('customer.csv', index=None)
json_customer

In [5]:
ls

DBs and Pandas.ipynb
DM sobre RS y SW.ipynb
Machine Learning with Scikit-learn.ipynb
Starting with pandas.ipynb
Starting with plots.ipynb
customer.csv
customer.json
data/


#### Hacer la query especificando el "engine" que se desea usar

In [6]:
df = pd.read_sql_query('select * from "Customer"',con=engine)
df

,idCustomer,name,addess,phone,birthday,gender
0,1,Customer 1,Address 1,11111111,2000-10-10,F
1,2,Customer 2,Address 2,22222222,2015-01-12,M
2,3,Customer 3,Address 3,33333333,2014-01-10,F
3,4,Customer 3,Address 4,44444444,2011-01-10,F


#### Hacer la query especificando el "engine" que se desea usar

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 6 columns):
idCustomer    4 non-null int64
name          4 non-null object
addess        4 non-null object
phone         4 non-null object
birthday      4 non-null datetime64[ns]
gender        4 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 224.0+ bytes


In [8]:
df.describe()

,idCustomer
count,4.000000
mean,2.500000
std,1.290994
min,1.000000
25%,1.750000
50%,2.500000
75%,3.250000
max,4.000000


In [ ]:
df

Link a [Pandas NB](Starting%20with%20pandas.ipynb) para ver join, merge, append, etc

#### Agregando un nuevo registro a nuestra tabla "Customer" con pandas

In [ ]:
new_df = pd.DataFrame([[4, pd.datetime(2011, 1, 10), 'F']], columns=df.columns)
new_df

In [ ]:
new_df.to_sql('Customer', engine, if_exists='append', index=None)

#### Agregando una nueva tabla a nuestra db desde pandas

In [ ]:
new_table = pd.DataFrame([], columns=['WineCode', 'Type', 'Vintage'])
new_table

In [ ]:
new_table.to_sql('Wine', engine, index=None)

#### Ahora hagamos lo mismo con sqlalchemy

In [ ]:
from sqlalchemy import MetaData, types
from sqlalchemy import Table, Column

In [ ]:
metadata = MetaData()

In [ ]:
time = Table('Time', metadata,
    Column('TimeCode', types.Integer, primary_key=True),
    Column('Date', types.DateTime, nullable=False),
 )

In [ ]:
metadata.create_all(engine)

### Agregando elementos a Wine con pandas

In [ ]:
data = [[1, 'White', 2000],
        [2, 'red', 2015],
        [3, 'rose', 2014]]

In [ ]:
new_df = pd.DataFrame(data, columns=df_wine.columns)

In [ ]:
new_df.to_sql('Wine', engine, if_exists='append', index=None)

In [ ]:
df_wine = pd.read_sql_query('select * from "Wine"',con=engine)
df_wine

#### Hacemos un append de un CSV

In [ ]:
pd.read_json?

In [32]:
json = pd.read_json('./customer.json')
json.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4 entries, 0.0 to 3.0
Data columns (total 6 columns):
addess        4 non-null object
birthday      4 non-null int64
gender        4 non-null object
idCustomer    4 non-null int64
name          4 non-null object
phone         4 non-null int64
dtypes: int64(3), object(3)
memory usage: 224.0+ bytes


#### Convierto la fecha

In [33]:
pd.to_datetime(json['birthday'], unit='ns')

0   2000-10-10
1   2015-01-12
2   2014-01-10
3   2011-01-10
Name: birthday, dtype: datetime64[ns]

In [34]:
json['birthday']

0     971136000000000000
1    1421020800000000000
2    1389312000000000000
3    1294617600000000000
Name: birthday, dtype: int64

In [36]:
json['birthday'] = pd.to_datetime(json['birthday'], unit='ns')
json


,addess,birthday,gender,idCustomer,name,phone
0,Address 1,2000-10-10,F,1,Customer 1,11111111
1,Address 2,2015-01-12,M,2,Customer 2,22222222
2,Address 3,2014-01-10,F,3,Customer 3,33333333
3,Address 4,2011-01-10,F,4,Customer 3,44444444


In [13]:
pd.__version__

'0.16.2'

In [65]:
csv = pd.read_csv('./customer.csv')
csv['birthday'] = pd.to_datetime(csv['birthday'], unit='ns')
csv.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 5
Data columns (total 6 columns):
idCustomer    6 non-null int64
name          6 non-null object
addess        6 non-null object
phone         6 non-null int64
birthday      6 non-null datetime64[ns]
gender        6 non-null object
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 336.0+ bytes


In [ ]:
df_customer

#### Hacemos un merge entre el JSON y CSV

In [67]:
merged = pd.merge(json, csv, how='outer')
merged

,addess,birthday,gender,idCustomer,name,phone
0,Address 1,2000-10-10,F,1,Customer 1,11111111
1,Address 2,2015-01-12,M,2,Customer 2,22222222
2,Address 3,2014-01-10,F,3,Customer 3,33333333
3,Address 4,2011-01-10,F,4,Customer 3,44444444
4,Address 4,2011-01-10,F,5,Customer 5,44444444
5,Address 4,2011-01-10,F,6,Customer 6,44444444
